# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [ ]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

from google.colab import drive
#drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/S6APP7/GRO620-E22

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


/content/gdrive/MyDrive/S6APP7/GRO620-E22


## Acquisition et caractéristiques de la lumière

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

*(réponse ici)*

CAMERA BODY\
Optics -> Lentille (Distance focale)\
Aperture -> Trou (Quantité de lumière)\
Shutter -> Ouverture/Fermeture (Temps d'exposition)\

SENSOR CHIP\
Sensor (CMOS/CCD) -> Matrice de capteurs (Pixels)\
Gain (ISO) -> Amplifie le signal (Mais le bruit aussi)\
A/D -> Convertisseur analogue-digital\
RAW -> Sortie directe de la caméra (Pas de perte)\

DSP\
Demosaic -> Combiner RGB (Pattern Bayer)\
(Sharpen) -> Ajoute des contrastes suite au flou du Demosaic\
White balance -> Balancer les couleurs par rapport au blanc\
Gamma/Curve -> Transforme la clareté linéaire sur une courbe exponentielle\
Compress -> Élimine de l'information redondante préférablement\
JPEG -> Sortie traitée et compressée de la caméra (Pertes)

### Q1.2

Quelle est la différence entre paramètres intrinsèques et extrinsèques du caméra ? Décrivez chaque type en une phrase.

*(réponse ici)*

Intrinsèques: Propre à la caméra\
-Résolution\
-Distance focal

Extrinsèques: Externe à la caméra\
-Référentiel de la caméra

### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ?

In [7]:
# Réponse ici.
K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]])

f = K[0, 0]*((30/2)/K[0, 2])
print(f)

9.08203125


### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement.

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

*(réponse ici)*

Tolérances sur l'assemblage\
Imperfections de la lentille\
Tolérances sur le capteur

Tolérances sur la position de la caméra

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

*(réponse ici)*

Calibration

### Q1.5

Dans cette image synthétique : 

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

*(réponse ici)*

Illumination diffuse = Couleur du matériel\
Reflets spéculaires = Effet miroir (Points blancs)

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Vous pouvez répondre en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

*(réponse ici)*

Géométrie de la scène (Triangulation avec les 2 reflets sur les 2 sphères)


### Q1.6

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

*(réponse ici)*

Calibration différente

**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

*(réponse ici)*

Changement de repère vers un repère commun à l'aide du white balance

## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [48]:
T_01 = np.array([
                 [0, 1, 0, 240],
                 [1, 0, 0, 80],
                 [0, 0, -1, 120],
                 [0, 0, 0, 1]
])
print("T_01:\n", T_01)

T_01:
 [[  0   1   0 240]
 [  1   0   0  80]
 [  0   0  -1 120]
 [  0   0   0   1]]


**b)** Trouvez maintenant la transformation inverse.

In [49]:
T_10 = np.linalg.inv(T_01)
print("T_10:\n", T_10)

T_10:
 [[   0.    1.    0.  -80.]
 [   1.    0.    0. -240.]
 [  -0.   -0.   -1.  120.]
 [   0.    0.    0.    1.]]


**c)** Soit le point $p_0 = [8, 5, 1]^T$, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [50]:
p_0 = [8, 5, 1]
p_1 = [0, 0, 0]

p_1 = (T_10@[p_0[0], p_0[1], p_0[2], 1])[0:3]
print("p_1:\n", p_1)

p_1:
 [ -75. -232.  119.]


### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$.

In [51]:
print(K) # Si vous n'avez pas réutilisé la variable K, elle aura toujours la même valeur qu'à la question Q1.3.

[[6.200e+02 0.000e+00 1.024e+03]
 [0.000e+00 6.200e+02 5.120e+02]
 [0.000e+00 0.000e+00 1.000e+00]]


**b)** Soit le point $p_0 = [0.250, 0.010, 0.000]$. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra.

In [58]:
p_0 = np.array([250., 10., 0.])
Ka = np.zeros((4, 4))
Ka[3, 3] = 1
Ka[0:3, 0:3] = K
print(Ka)

P = Ka@T_10
print(P)

xsw = P @ [p_0[0], p_0[1], p_0[2], 1]
print(xsw)

xs = xsw/xsw[2]
print(xs)

[[6.200e+02 0.000e+00 1.024e+03 0.000e+00]
 [0.000e+00 6.200e+02 5.120e+02 0.000e+00]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00]]
[[ 0.000e+00  6.200e+02 -1.024e+03  7.328e+04]
 [ 6.200e+02  0.000e+00 -5.120e+02 -8.736e+04]
 [ 0.000e+00  0.000e+00 -1.000e+00  1.200e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]
[7.948e+04 6.764e+04 1.200e+02 1.000e+00]
[6.62333333e+02 5.63666667e+02 1.00000000e+00 8.33333333e-03]


## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [ ]:
l_conv = 0

### Q3.2

Soit le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [ ]:
x_s = np.array([120,200])